In [3]:
from pathlib import Path
import pandas as pd, numpy as np, re

BASE = Path("/Users/kartik/Desktop/aidev-phase2")
DATA = BASE / "data"
OUT  = BASE / "outputs"
OUT.mkdir(exist_ok=True, parents=True)

def load_csv(p):
    # robust loader; prevents dtype warnings
    df = pd.read_csv(p, low_memory=False)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    print(f"Loaded {p.name}: {len(df)} rows, {len(df.columns)} cols")
    return df

# paths
PULLS   = DATA / "all_pull_requests" / "all_pull_request.csv"
REPOS   = DATA / "all_repositories" / "all_repository.csv"
COMMITS = DATA / "pr_commit_details" / "pr_commit_details.csv"
REVCMTS = DATA / "pr_review_comments" / "pr_review_comments.csv"
COMMENTS = DATA / "pr_comments" / "pr_comments.csv"

pr        = load_csv(PULLS)
diff      = load_csv(COMMITS)
rev       = load_csv(REVCMTS)
repo      = load_csv(REPOS)
pr_comnts = load_csv(COMMENTS)


Loaded all_pull_request.csv: 932791 rows, 14 cols
Loaded pr_commit_details.csv: 711923 rows, 14 cols
Loaded pr_review_comments.csv: 19450 rows, 15 cols
Loaded all_repository.csv: 116211 rows, 7 cols
Loaded pr_comments.csv: 39122 rows, 7 cols


In [4]:
def smart_rename(df, mapping):
    """mapping: {'target_col': ['candidate1','candidate2',...]}"""
    for target, candidates in mapping.items():
        for c in candidates:
            if c in df.columns:
                if target != c:
                    df.rename(columns={c: target}, inplace=True)
                break  # stop at first match

# PR table
smart_rename(pr, {
    'pr_id':        ['pr_id','pull_request_id','number','id'],
    'repo_id':      ['repo_id','repository_id'],
    'author_type':  ['author_type','contributor_type','creator_type'],
    'title':        ['title'],
    'body':         ['body','description'],
    'state':        ['state','status'],
    'created_at':   ['created_at','created'],
    'merged_at':    ['merged_at','merged'],
    'closed_at':    ['closed_at','closed'],
})

# Commit/diff table
smart_rename(diff, {
    'pr_id':         ['pr_id','pull_request_id','number','id'],
    'lines_added':   ['lines_added','additions','added'],
    'lines_deleted': ['lines_deleted','deletions','deleted','removed'],
    'file_path':     ['file_path','filepath','file','filename','path','new_path','old_path']
})

# Review comments table
smart_rename(rev, {
    'pr_id':        ['pr_id','pull_request_id','number','id'],
    'comment_body': ['comment_body','body','text','comment'],
    'created_at':   ['created_at','created']
})

# Repository table
smart_rename(repo, {
    'repo_id':    ['repo_id','id'],
    'stars':      ['stargazers_count','stars'],
    'forks':      ['forks_count','forks']
})

# ensure expected columns exist
print("PR columns:", pr.columns.tolist())
print("DIFF columns:", diff.columns.tolist())
print("REV columns:", rev.columns.tolist())
print("REPO columns:", repo.columns.tolist())


PR columns: ['id', 'pr_id', 'title', 'user', 'user_id', 'state', 'created_at', 'closed_at', 'merged_at', 'repo_url', 'repo_id', 'html_url', 'body', 'agent']
DIFF columns: ['sha', 'pr_id', 'author', 'committer', 'message', 'commit_stats_total', 'commit_stats_additions', 'commit_stats_deletions', 'file_path', 'status', 'lines_added', 'lines_deleted', 'changes', 'patch']
REV columns: ['pr_id', 'pull_request_review_id', 'user', 'user_type', 'diff_hunk', 'path', 'position', 'original_position', 'commit_id', 'original_commit_id', 'comment_body', 'pull_request_url', 'created_at', 'updated_at', 'in_reply_to_id']
REPO columns: ['repo_id', 'url', 'license', 'full_name', 'language', 'forks', 'stars']


In [5]:
for col in ['lines_added','lines_deleted']:
    if col in diff.columns:
        diff[col] = pd.to_numeric(diff[col], errors='coerce').fillna(0).astype(int)

if 'file_path' in diff.columns:
    diff['file_path'] = diff['file_path'].astype(str)


In [6]:
# choose a file-identity column for counting touched files
file_key = 'file_path' if 'file_path' in diff.columns else None

agg_diff_specs = {
    'loc_added':   ('lines_added','sum')   if 'lines_added'   in diff.columns else (None, None),
    'loc_deleted': ('lines_deleted','sum') if 'lines_deleted' in diff.columns else (None, None),
    'files_touched': (file_key, 'nunique') if file_key else ('pr_id','size')
}

# build aggregation dict only for available columns
agg_dict = {}
if agg_diff_specs['loc_added'][0]:   agg_dict['loc_added']   = agg_diff_specs['loc_added'][1]
if agg_diff_specs['loc_deleted'][0]: agg_dict['loc_deleted'] = agg_diff_specs['loc_deleted'][1]
# Pandas needs mapping per column name → function; do via .agg on a dict of Series
grouped = diff.groupby('pr_id')
agg_pieces = {}
if 'lines_added' in diff.columns:
    agg_pieces['loc_added'] = grouped['lines_added'].sum()
if 'lines_deleted' in diff.columns:
    agg_pieces['loc_deleted'] = grouped['lines_deleted'].sum()
if file_key:
    agg_pieces['files_touched'] = grouped[file_key].nunique()
else:
    agg_pieces['files_touched'] = grouped['pr_id'].size()

agg_diff = pd.DataFrame(agg_pieces).reset_index()

# review comments aggregation (robust)
agg_rev = (rev.groupby('pr_id')
             .agg(n_review_comments=('pr_id','size'),
                  first_comment_at=('created_at','min'))
             .reset_index())

# merge to full
full = (pr
        .merge(agg_diff, on='pr_id', how='left')
        .merge(agg_rev,  on='pr_id', how='left'))

if not repo.empty and 'repo_id' in full.columns and set(['stars','forks']).issubset(repo.columns):
    full = full.merge(repo[['repo_id','stars','forks']], on='repo_id', how='left')

# fill NAs for numeric aggregates
for col in ['loc_added','loc_deleted','files_touched','n_review_comments','stars','forks']:
    if col in full.columns:
        full[col] = pd.to_numeric(full[col], errors='coerce').fillna(0)

full.to_csv(OUT/'full_join.csv', index=False)
print("✅ Saved:", OUT/'full_join.csv', "rows:", len(full))


✅ Saved: /Users/kartik/Desktop/aidev-phase2/outputs/full_join.csv rows: 932791


In [7]:
# Acceptance label
state = full.get('state','').astype(str).str.lower()
full['y_accept'] = np.where(state.eq('merged'), 1,
                    np.where(state.eq('closed'), 0, np.nan))

# Time to merge
def to_hours(row):
    try:
        if pd.notna(row['merged_at']) and pd.notna(row['created_at']):
            return (pd.to_datetime(row['merged_at']) - pd.to_datetime(row['created_at'])).total_seconds()/3600.0
    except Exception: pass
    return np.nan

if {'created_at','merged_at'}.issubset(full.columns):
    full['hours_to_merge'] = full.apply(to_hours, axis=1)

print(full[['y_accept','hours_to_merge']].describe(include='all'))


       y_accept  hours_to_merge
count  859927.0   790139.000000
mean        0.0        2.751494
std         0.0       31.554555
min         0.0        0.000278
25%         0.0        0.003611
50%         0.0        0.009167
75%         0.0        0.051667
max         0.0     2546.475556


In [8]:
import pandas as pd, numpy as np
from pathlib import Path

OUT = Path("/Users/kartik/Desktop/aidev-phase2/outputs")
full = pd.read_csv(OUT/"full_join.csv", low_memory=False)

print("State sample:", full['state'].astype(str).str.lower().value_counts().head(10))
print("# merged_at notna:", full['merged_at'].notna().sum())
print("# closed_at notna:", full['closed_at'].notna().sum())
print("# created_at notna:", full['created_at'].notna().sum())


State sample: state
closed    859927
open       72864
Name: count, dtype: int64
# merged_at notna: 790139
# closed_at notna: 859927
# created_at notna: 932791


In [9]:
# normalize state text
state_norm = full['state'].astype(str).str.lower()

# acceptance label:
# 1 if merged_at exists
# 0 if closed (state says closed/declined) OR closed_at exists but merged_at is null
# NaN if still open (state=open OR both closed_at/merged_at null)
full['y_accept'] = np.where(full['merged_at'].notna(), 1,
                    np.where(
                        (state_norm.isin(['closed','declined','rejected'])) | 
                        ((full['closed_at'].notna()) & (full['merged_at'].isna())),
                        0,
                        np.nan
                    ))

# recompute hours_to_merge using timestamps robustly
def to_hours(row):
    try:
        ca = pd.to_datetime(row['created_at'])
        ma = pd.to_datetime(row['merged_at'])
        if pd.notna(ca) and pd.notna(ma):
            return (ma - ca).total_seconds() / 3600.0
    except Exception:
        pass
    return np.nan

full['hours_to_merge'] = full.apply(to_hours, axis=1)

full[['y_accept','hours_to_merge']].describe(include='all')
full['y_accept'].value_counts(dropna=False)


y_accept
1.0    790139
NaN     72864
0.0     69788
Name: count, dtype: int64

In [10]:
# contrib_type from 'agent' column (which may be bool, 0/1, "true"/"false")
s = full['agent'].astype(str).str.strip().str.lower()
is_agent = s.isin(['true','1','yes'])
is_human = s.isin(['false','0','no'])

full['contrib_type'] = np.where(is_agent, 'agent',
                         np.where(is_human, 'human', 'unknown'))

full['contrib_type'].value_counts(dropna=False)


contrib_type
unknown    932791
Name: count, dtype: int64

In [11]:
# If later you want 'hybrid' via comment text hints
if 'comment_body' in full.columns:
    hybrid_hint = full['comment_body'].astype(str).str.contains(r'\b(agent|copilot|ai)\b', case=False, regex=True)
    full.loc[(full['contrib_type']=='human') & hybrid_hint, 'contrib_type'] = 'hybrid'


In [12]:
full.to_csv(OUT/"modeling_table.csv", index=False)
print("✅ Saved corrected:", OUT/"modeling_table.csv")

print("Label distribution (including NaN):")
print(full['y_accept'].value_counts(dropna=False).to_string())

print("Contrib types:")
print(full['contrib_type'].value_counts(dropna=False).to_string())


✅ Saved corrected: /Users/kartik/Desktop/aidev-phase2/outputs/modeling_table.csv
Label distribution (including NaN):
y_accept
1.0    790139
NaN     72864
0.0     69788
Contrib types:
contrib_type
unknown    932791


In [13]:
# sanity cross-tab
pd.crosstab(full['y_accept'], full['state'].astype(str).str.lower()).head()


state,closed
y_accept,
0.0,69788
1.0,790139


In [14]:
full['agent'].value_counts(dropna=False).head(20)


agent
OpenAI_Codex    814522
Copilot          50447
Cursor           32941
Devin            29744
Claude_Code       5137
Name: count, dtype: int64

In [15]:
# Convert and normalize agent column
s = full['agent'].astype(str).str.strip().str.lower()

# AI keywords (from your dataset)
ai_tools = ['codex', 'copilot', 'cursor', 'devin', 'claude', 'openai', 'ai']

def classify_agent(val):
    if any(tool in val for tool in ai_tools):
        return 'agent'
    elif val in ['nan', 'none', '', 'null']:
        return 'human'
    else:
        return 'human'  # fallback for unknown cases

full['contrib_type'] = s.apply(classify_agent)
full['contrib_type'].value_counts(dropna=False)


contrib_type
agent    932791
Name: count, dtype: int64

In [16]:
from pathlib import Path
import pandas as pd, numpy as np, re

BASE = Path("/Users/kartik/Desktop/aidev-phase2")
DATA = BASE / "data"
OUT  = BASE / "outputs"
OUT.mkdir(exist_ok=True, parents=True)

# 1) Load the human PR file
HUMAN_PR_PATH = DATA / "human_pull_request" / "human_pull_request.csv"
human_df = pd.read_csv(HUMAN_PR_PATH, low_memory=False)
human_df.columns = [c.strip().lower().replace(" ", "_") for c in human_df.columns]
print("Loaded human PRs:", HUMAN_PR_PATH, human_df.shape)

# 2) Normalize columns to match your PR schema
# PR columns you already have in the agent 'pr' table:
# ['id','pr_id','title','user','user_id','state','created_at','closed_at','merged_at','repo_url','repo_id','html_url','body','agent']
rename_map = {
    'pull_request_id': 'pr_id',
    'number':          'pr_id',
    'id':              'id',
    'repository_id':   'repo_id',
    'repo':            'repo_id',
    'status':          'state',
    'description':     'body',
    # timestamps:
    'created':         'created_at',
    'closed':          'closed_at',
    'merged':          'merged_at',
}
for src, tgt in rename_map.items():
    if src in human_df.columns and tgt not in human_df.columns:
        human_df.rename(columns={src: tgt}, inplace=True)

# Ensure the minimal set exists
for col in ['pr_id','repo_id','title','body','state','created_at','merged_at','closed_at']:
    if col not in human_df.columns:
        human_df[col] = np.nan

# 3) Tag as human (your agent file had explicit 'agent' names; here we force human)
human_df['agent'] = 'human'            # for consistency with your agent column
human_df['contrib_type'] = 'human'     # explicit contributor type

# 4) Build/refresh diff & review aggregations across the GLOBAL diff/rev tables you already loaded
#    (If those tables also include human PRs, they will join; if not, they’ll remain NaN which we fill to 0.)
# Recompute aggregations to be safe (uses robust names from your DIFF/REV printouts):
agg_diff = (diff
            .groupby('pr_id')
            .agg(loc_added=('lines_added','sum'),
                 loc_deleted=('lines_deleted','sum'),
                 files_touched=('file_path','nunique'))
            .reset_index())

agg_rev = (rev
           .groupby('pr_id')
           .agg(n_review_comments=('pr_id','size'),
                first_comment_at=('created_at','min'))
           .reset_index())

# 5) Build a unified PR table: agent PRs (from full) + human PRs, then re-merge aggregations
#    If you still have the earlier 'full' (agent-only) in memory, great.
#    Otherwise reload it:
agent_full = pd.read_csv(OUT / "full_join.csv", low_memory=False)

# Strip down agent_full to PR-level cols (to avoid double-merging old aggregates)
pr_agent_cols = ['pr_id','repo_id','title','body','state','created_at','merged_at','closed_at','agent']
for c in pr_agent_cols:
    if c not in agent_full.columns:
        agent_full[c] = np.nan
pr_agent = agent_full[pr_agent_cols].copy()

# Stack agent + human PRs
pr_all = pd.concat([pr_agent, human_df[pr_agent_cols]], ignore_index=True)

# Re-merge diff/review/repo so both agent + human get features
combined = (pr_all
            .merge(agg_diff, on='pr_id', how='left')
            .merge(agg_rev,  on='pr_id', how='left'))

if not repo.empty and set(['repo_id','stars','forks']).issubset(repo.columns):
    combined = combined.merge(repo[['repo_id','stars','forks']], on='repo_id', how='left')

# Fill numeric NAs
for col in ['loc_added','loc_deleted','files_touched','n_review_comments','stars','forks']:
    if col in combined.columns:
        combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0)

# 6) Labels (acceptance + effort) — timestamp-first logic
state_norm = combined['state'].astype(str).str.lower()
combined['y_accept'] = np.where(combined['merged_at'].notna(), 1,
                        np.where(
                            (state_norm.isin(['closed','declined','rejected'])) |
                            ((combined['closed_at'].notna()) & (combined['merged_at'].isna())),
                            0, np.nan))

def to_hours(row):
    try:
        ca = pd.to_datetime(row['created_at'])
        ma = pd.to_datetime(row['merged_at'])
        if pd.notna(ca) and pd.notna(ma):
            return (ma - ca).total_seconds()/3600.0
    except Exception:
        pass
    return np.nan

combined['hours_to_merge'] = combined.apply(to_hours, axis=1)

# 7) contrib_type from 'agent' column (agent tools vs human)
s = combined['agent'].astype(str).str.strip().str.lower()
ai_tools = ['codex','copilot','cursor','devin','claude','openai','ai']

def classify_agent(val):
    if any(tool in val for tool in ai_tools):
        return 'agent'
    elif val in ['human','nan','','none','null']:
        return 'human'
    else:
        # be conservative: if not a known AI tool name, treat as human
        return 'human'

combined['contrib_type'] = s.apply(classify_agent)

print("\nLabel distribution (incl. NaN):")
print(combined['y_accept'].value_counts(dropna=False).to_string())

print("\nContributor types:")
print(combined['contrib_type'].value_counts(dropna=False).to_string())

# 8) Save final modeling table
combined.to_csv(OUT / "modeling_table_combined.csv", index=False)
print("\n✅ Saved:", OUT / "modeling_table_combined.csv")


Loaded human PRs: /Users/kartik/Desktop/aidev-phase2/data/human_pull_request/human_pull_request.csv (6618, 13)

Label distribution (incl. NaN):
y_accept
1.0    795220
NaN     73333
0.0     70856

Contributor types:
contrib_type
agent    932791
human      6618

✅ Saved: /Users/kartik/Desktop/aidev-phase2/outputs/modeling_table_combined.csv
